In [103]:
!pip install --root-user-action=ignore pandas
!pip install --root-user-action=ignore transformers
!pip install --root-user-action=ignore numpy
!pip install --root-user-action=ignore torch
!pip install --root-user-action=ignore tqdm
!pip install --root-user-action=ignore scikit-learn
!pip install --root-user-action=ignore gdown

In [118]:
import pandas as pd
import torch
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [105]:
# !mkdir ./prediction

In [106]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)
model_bertweet = AutoModel.from_pretrained("vinai/bertweet-base").to(device)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [107]:
model = UserStanceClassifier(input_dim=input_dim).to(device)
print(model)

UserStanceClassifier(
  (linear): Linear(in_features=779, out_features=2, bias=True)
)


In [114]:
class UserStanceClassifier(nn.Module):
    def __init__(self, input_dim):
        super(UserStanceClassifier, self).__init__()
        self.linear = nn.Linear(input_dim, 2)  # Assuming binary classification

    def forward(self, x):
        return self.linear(x)

def tokenize_and_embed(tweets, tokenizer, model, batch_size=32):
    model.to(device)  # Ensure the model is on the correct device
    all_embeddings = torch.empty((0, model.config.hidden_size), device=device)
    
    num_batches = len(tweets) // batch_size + (0 if len(tweets) % batch_size == 0 else 1)
    
    for i in tqdm(range(num_batches), desc="Generating embeddings"):
        batch_tweets = tweets[i*batch_size:(i+1)*batch_size]
        inputs = tokenizer(batch_tweets, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(1)
        all_embeddings = torch.cat((all_embeddings, batch_embeddings), dim=0)
    
    return all_embeddings.cpu()  # Move embeddings back to CPU for further processing or storage


def ensure_numeric(df, start_col=3, fillna_value=0):
    for col in df.columns[start_col:]:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(fillna_value)
    return df

def standard_scale_embeddings(embeddings):
    # Assuming embeddings is a 2D PyTorch tensor
    scaler = StandardScaler()
    scaled_embeddings = scaler.fit_transform(embeddings.cpu().numpy())
    return torch.tensor(scaled_embeddings, dtype=torch.float).to(embeddings.device)

def prepare_data(data, tokenizer, model, version='none', batch_size=32, device=None, emfd_embeddings=None, frameaxis_embeddings=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    tweets = data['tweet'].tolist()
    bertweet_embeddings = tokenize_and_embed(tweets, tokenizer, model, batch_size).to(device)
    bertweet_embeddings = standard_scale_embeddings(bertweet_embeddings)
    
    final_embeddings = bertweet_embeddings

    if version == 'emfd' and emfd_embeddings is not None:
        emfd_embeddings_processed = ensure_numeric(emfd_embeddings)
        emfd_values = emfd_embeddings_processed.iloc[:bertweet_embeddings.size(0), 3:].values
        emfd_tensor = torch.tensor(emfd_values, dtype=torch.float).to(device)
        emfd_tensor = standard_scale_embeddings(emfd_tensor)
        final_embeddings = torch.cat((final_embeddings, emfd_tensor), dim=1)
    elif version == 'frameaxis' and frameaxis_embeddings is not None:
        frameaxis_embeddings_processed = ensure_numeric(frameaxis_embeddings)
        frameaxis_values = frameaxis_embeddings_processed.iloc[:bertweet_embeddings.size(0), 3:].values
        frameaxis_tensor = torch.tensor(frameaxis_values, dtype=torch.float).to(device)
        frameaxis_tensor = standard_scale_embeddings(frameaxis_tensor)
        final_embeddings = torch.cat((final_embeddings, frameaxis_tensor), dim=1)

    labels = torch.tensor(data['stance'].iloc[:final_embeddings.size(0)].values, dtype=torch.long).to(device)
    
    dataset = TensorDataset(final_embeddings, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return dataloader

In [115]:
def train_model(dataloader, input_dim, learning_rate=2e-5, epochs=50, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UserStanceClassifier(input_dim=input_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    # print(input_dim)
    best_loss = float('inf')
    no_improvement = 0

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, labels = [b.to(device) for b in batch]
            # print(f"Input shape: {inputs.shape}")
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        print(f"Average Loss: {avg_loss}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improvement = 0
        else:
            no_improvement += 1
        
        if no_improvement >= patience:
            print("Early stopping triggered.")
            break

    return model

def predict(model, dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs in tqdm(dataloader):
            inputs = inputs[0].to(device)  # Assuming inputs are the first element
            outputs = model(inputs)
            _, predicted = torch.max(outputs, dim=1)
            predictions.extend(predicted.cpu().tolist())  # Move predictions to CPU
    return predictions

In [116]:
def evaluate_model(model, dataloader, device=None, save_predictions_to=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()  # Set the model to evaluation mode
    true_labels = []
    predictions = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating model"):
            inputs, labels = [b.to(device) for b in batch]
            outputs = model(inputs)
            _, preds = torch.max(outputs, dim=1)
            true_labels.extend(labels.cpu().numpy())
            predictions.extend(preds.cpu().tolist())

    # Compute evaluation metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

    # Save predictions to CSV if a filename is provided
    if save_predictions_to:
        prediction_df = pd.DataFrame({'True Labels': true_labels, 'Predictions': predictions})
        prediction_df.to_csv(save_predictions_to, index=False)

    return accuracy, precision, recall, f1

def evaluate_multiple_times(model, dataloader, target, mode, device=None, iterations=5):
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for i in tqdm(range(iterations), desc="Evaluations"):
        # Generate a unique filename for this iteration's predictions
        predictions_filename = f'./prediction/{target}_{mode}_predictions_iteration_{i+1}.csv'
        accuracy, precision, recall, f1 = evaluate_model(model, dataloader, device, save_predictions_to=predictions_filename)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate the average of each metric
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    return avg_accuracy, avg_precision, avg_recall, avg_f1_score

In [ ]:
targets = ['trump', 'mask', 'racial']
modes = ['emfd']

for target in targets:
    none_data_path = f'./tweet_level/{target}_train.csv'  # Path for the 'none' mode
    none_data = pd.read_csv(none_data_path)

    for mode in modes:
        # Adjust the data path based on mode
        if mode == 'none':
            data = none_data  # Use the data loaded for 'none' mode
        elif mode == 'emfd':
            emfd_path = f'./tweet_level/{target}_train_emfd.csv'
            emfd_data = pd.read_csv(emfd_path)
            input_dim = 768 + (emfd_data.shape[1] - 3)  # Correct calculation for input_dim
        elif mode == 'frameaxis':
            frameaxis_path = f'./tweet_level/{target}_train_frameaxis.csv'
            frameaxis_data = pd.read_csv(frameaxis_path)
            input_dim = 768 + (frameaxis_data.shape[1] - 3)

        # Prepare DataLoader for training
        dataloader = prepare_data(none_data, tokenizer, model_bertweet, version=mode, batch_size=32, device=device, emfd_embeddings=emfd_data if mode == 'emfd' else None, frameaxis_embeddings=frameaxis_data if mode == 'frameaxis' else None)
        
        # Train model
        model = train_model(dataloader, input_dim=input_dim, learning_rate=2e-5, epochs=50, patience=5)
        
        # Load and prepare test data
        test_data_path = f'./tweet_level/{target}_test.csv' if mode == 'none' else f'./tweet_level/{target}_test_{mode}.csv'
        test_data = pd.read_csv(test_data_path)
        test_dataloader = prepare_data(test_data, tokenizer, model_bertweet, version=mode, batch_size=32, device=device, emfd_embeddings=emfd_data if mode == 'emfd' else None, frameaxis_embeddings=frameaxis_data if mode == 'frameaxis' else None)
        
        # Evaluate model
                # Prepare DataLoader for testing
        # test_dataloader = prepare_data(test_data, tokenizer, model_bertweet, version=mode, batch_size=32)
        
        # Evaluate model
        # iterations = 5
        accuracy, precision, recall, f1 = evaluate_multiple_times(model, test_dataloader, target, mode, device=device)
        # accuracy, precision, recall, f1 = evaluate_multiple_times(model, test_dataloader, device=device)
        
        print(f'CB-{mode}-{target}:')
        print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

Epoch 1/50: 100%|██████████| 6896/6896 [00:07<00:00, 885.78it/s] 


Average Loss: 0.5919445250010477


Epoch 2/50: 100%|██████████| 6896/6896 [00:07<00:00, 931.96it/s] 


Average Loss: 0.5061422259548978


Epoch 3/50: 100%|██████████| 6896/6896 [00:07<00:00, 958.00it/s] 


...

Epoch 49/50: 100%|██████████| 16700/16700 [00:16<00:00, 999.47it/s] 


Average Loss: 0.4502139671182561


Epoch 50/50: 100%|██████████| 16700/16700 [00:17<00:00, 960.34it/s] 


Average Loss: 0.45016244088104385


Evaluating model:  83%|████████▎ | 13262/15961 [00:06<00:01, 1584.07it/s]